In [1]:
from __future__ import print_function, division

import matplotlib.pyplot as plt

import sys

import keras

from sklearn import preprocessing
from sklearn.cross_validation import train_test_split

import time
from tqdm import tqdm

from imutils import paths
from numpy import *
import numpy as np

from matplotlib.colors import Normalize

import argparse
import imutils,sklearn
import os, cv2, re, random, shutil, imageio, pickle

%matplotlib inline  

/home/huiqy/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/huiqy/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#灰度共生矩阵
#定义最大灰度级数
gray_level = 16

def maxGrayLevel(img):
    max_gray_level=0
    (height,width)=img.shape
    for y in range(height):
        for x in range(width):
            if img[y][x] > max_gray_level:
                max_gray_level = img[y][x]
    return max_gray_level+1

def getGlcm(img,d_x,d_y):
    srcdata=img.copy()
    ret=[[0.0 for i in range(gray_level)] for j in range(gray_level)]
    (height,width) = img.shape

    max_gray_level=maxGrayLevel(img)

    #若灰度级数大于gray_level，则将图像的灰度级缩小至gray_level，减小灰度共生矩阵的大小
    if max_gray_level > gray_level:
        for j in range(height):
            for i in range(width):
                srcdata[j][i] = srcdata[j][i]*gray_level / max_gray_level

    for j in range(height-d_y):
        for i in range(width-d_x):
            rows = srcdata[j][i]
            cols = srcdata[j + d_y][i+d_x]
            ret[rows][cols]+=1.0

    for i in range(gray_level):
        for j in range(gray_level):
            ret[i][j]/=float(height*width)

    return np.array(ret)
    
def feature_computer(p):
    Con=0.0
    Eng=0.0
    Asm=0.0
    Idm=0.0
    for i in range(gray_level):
        for j in range(gray_level):
            Con+=(i-j)*(i-j)*p[i][j]
            Asm+=p[i][j]*p[i][j]
            Idm+=p[i][j]/(1+(i-j)*(i-j))
            if p[i][j]>0.0:
                Eng+=p[i][j]*math.log(p[i][j])
    return Asm,Con,-Eng,Idm

def batch_GLCM(images):
    greycomatrix_list = []
    for i in tqdm(range(len(images))):
        img = (images.astype(np.float32)* 255)[i, :, :, :].astype(np.uint8)
        gray=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        glcm_0=getGlcm(gray, 1,0)
        asm0,con0,eng0,idm0 = feature_computer(glcm_0)
        
        glcm_1=getGlcm(gray, 0,1)
        asm1,con1,eng1,idm1 = feature_computer(glcm_1)
        
        glcm_2=getGlcm(gray, 1,1)
        asm2,con2,eng2,idm2 = feature_computer(glcm_2)

        greycomatrix_list.append([asm0,con0,eng0,idm0,
                                  asm1,con1,eng1,idm1,
                                  asm2,con2,eng2,idm2])
        
    greycomatrix_list=np.array(greycomatrix_list,dtype = float32)
    
    print (greycomatrix_list.shape)
    return greycomatrix_list

In [3]:
def batch_canny_rgb(images):
    canny_list = []
    for i in tqdm(range(len(images))):
        img = (images.astype(np.float32)* 255)[i, :, :, :].astype(np.uint8)
        img = cv2.resize(img, (32,32))
        canny_superposition = np.zeros(shape=(img.shape[0],img.shape[1]),dtype=np.uint16)
        for channel in range(3):
            img_ch = img[:,:,channel]
            canny_ch = cv2.Canny(img_ch, 60, 150)
            canny_superposition =  canny_superposition + (canny_ch).astype(np.uint16)
        canny_superposition = cv2.normalize(canny_superposition, canny_superposition,0,255,cv2.NORM_MINMAX)
        canny_list.append(np.array((canny_superposition).astype(np.uint8)).flatten())
    canny_list = np.array(canny_list,dtype = float32)
    
    print (canny_list.shape)
    return canny_list

def batch_canny_gray(images):
    canny_list = []
    for i in tqdm(range(len(images))):
        img = (images.astype(np.float32)* 255)[i, :, :, :].astype(np.uint8)
        img = cv2.resize(img, (32,32))
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        canny = cv2.Canny(gray, 60, 150)
        canny = cv2.normalize(canny, canny,0,255,cv2.NORM_MINMAX)
        canny_list.append(np.array((canny).astype(np.uint8)).flatten())
    canny_list = np.array(canny_list,dtype = float32)
    
    print (canny_list.shape)
    return canny_list

In [4]:
#Pyramid
def batch_pyramid(images):
    pyramid_list = []
    for i in tqdm(range(len(images))):
        img = (images.astype(np.float32)* 255)[i, :, :, :].astype(np.uint8)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        G = img.copy()
        gp = [G]
        for i in range(4):
            G = cv2.pyrDown(G)
            gp.append(G)
        lp = [gp[4]]
        for i in range(4,0,-1):
            GE = cv2.pyrUp(gp[i])
            (x,y) = gp[i-1].shape
            L = cv2.subtract(gp[i-1],GE[:x,:y])
            lp.append(L)
        lp_1 = cv2.normalize(lp[1], lp[1],0,255,cv2.NORM_MINMAX)
        lp_2 = cv2.normalize(lp[2], lp[2],0,255,cv2.NORM_MINMAX)
        v_1 = np.array((lp_1).astype(np.uint8)).flatten() 
        v_2 = np.array((lp_2).astype(np.uint8)).flatten() 
        v_1_2 = np.hstack((v_1,v_2))
        pyramid_list.append(np.array((v_1_2).astype(np.uint8)).flatten())
    pyramid_list = np.array(pyramid_list,dtype = float32)
    
    print (pyramid_list.shape)
    return pyramid_list

In [5]:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split('(\d+)', text) ]

def load_flower_data():
    # grab the list of images that we'll be describing
    print("[INFO] handling images...")
    TRAIN_ORIGINAL_DIR = '../train/'
    TRAIN_SUB_DIR = '../subsample/'
    TRAIN_GAN = '../image_gan/'
    TEST_DIR = '../test/'

    # use this for full dataset
    train_images = [TRAIN_SUB_DIR + i for i in os.listdir(TRAIN_SUB_DIR)]
    test_images = [TEST_DIR + i for i in os.listdir(TEST_DIR)]
    
    train_images.sort(key=natural_keys)
    test_images.sort(key=natural_keys)

    # initialize the features matrix and labels list
    trainImage = []
    trainLabels = []
    testImage = []
    testLabels = []

    # loop over the input images
    for (i, imagePath) in enumerate(train_images):
        # extract the class label
        # get the labels from the name of the images by extract the string before "_"
        label = imagePath.split(os.path.sep)[-1].split("_")[0]

        # read and resize image
        img = cv2.imread(imagePath)
        img = cv2.resize(img, (128,128))

        # add the messages we got to features and labels matricies
        trainImage.append(img)
        trainLabels.append(label)

        # show an update every 100 images until the last image
        if i > 0 and ((i + 1) % 1000 == 0 or i == len(train_images) - 1):
            print("[INFO] processed {}/{}".format(i + 1, len(train_images)))
            
      # loop over the input images
    for (i, imagePath) in enumerate(test_images):
        # extract the class label
        # our images were named as labels.image_number.format
        # get the labels from the name of the images by extract the string before "_"
        label = imagePath.split(os.path.sep)[-1].split("_")[0]

        # extract CNN features in the image
        img = cv2.imread(imagePath)
        img = cv2.resize(img, (128,128))

        # add the messages we got to features and labels matricies
        testImage.append(img)
        testLabels.append(label)

        # show an update every 100 images until the last image
        if i > 0 and ((i + 1) % 1000 == 0 or i == len(test_images) - 1):
            print("[INFO] processed {}/{}".format(i + 1, len(test_images)))


    trainImage = np.array(trainImage,dtype = float32)
    trainLabels = np.array(trainLabels)
    testImage = np.array(testImage,dtype = float32)
    testLabels = np.array(testLabels)
    print (trainImage.shape)
    
    trainImage = trainImage.astype(np.float32) / 255
    testImage = testImage.astype(np.float32) / 255
    
    le = preprocessing.LabelEncoder()
    le.fit(trainLabels)
    list(le.classes_)
    trainLabels = le.transform(trainLabels) 
    testLabels = le.transform(testLabels) 
    
    return trainImage, trainLabels, testImage, testLabels

In [6]:
trainImage, trainLabels, testImage, testLabels = load_flower_data()

[INFO] handling images...
[INFO] processed 1000/2000
[INFO] processed 2000/2000
[INFO] processed 158/158
(2000, 128, 128, 3)


In [7]:
np.save('./trainLabels.npy', trainLabels)
np.save('./testLabels.npy', testLabels)

print (trainLabels)
print (trainLabels.shape)
print (testLabels)
print (testLabels.shape)

print("[INFO] trainLabels matrix: {:.4f}MB".format(
    (trainLabels.nbytes) / (1024 * 1000.0)))
print("[INFO] testLabels matrix: {:.4f}MB".format(
    (testLabels.nbytes) / (1024 * 1000.0)))


[0 1]
(2,)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1]
(158,)
[INFO] trainLabels matrix: 0.0000MB
[INFO] testLabels matrix: 0.0012MB


In [8]:
# Convert class vectors to binary class matrices.
nb_classes = 2
trainLabels_onehot = keras.utils.to_categorical(trainLabels, nb_classes)
testLabels_onehot = keras.utils.to_categorical(testLabels, nb_classes)

np.save('./trainLabels_onehot.npy', trainLabels_onehot)
np.save('./testLabels_onehot.npy', testLabels_onehot)

print (trainLabels_onehot)
print (trainLabels_onehot.shape)
print (testLabels_onehot)
print (testLabels_onehot.shape)

print("[INFO] trainLabels_onehot matrix: {:.4f}MB".format(
    (trainLabels_onehot.nbytes) / (1024 * 1000.0)))
print("[INFO] testLabels_onehot matrix: {:.4f}MB".format(
    (testLabels_onehot.nbytes) / (1024 * 1000.0)))

[[1. 0.]
 [0. 1.]]
(2, 2)
[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [

In [9]:
trainImage_canny = batch_canny_rgb(trainImage)
testImage_canny = batch_canny_rgb(testImage)

scaler = sklearn.preprocessing.StandardScaler().fit(trainImage_canny)
trainImage_canny = scaler.transform(trainImage_canny)
testImage_canny = scaler.transform(testImage_canny)

np.save('./trainImage_canny.npy', trainImage_canny)
np.save('./testImage_canny.npy', testImage_canny)

print("[INFO] trainImage_canny matrix: {:.2f}MB".format(
    (trainImage_canny.nbytes) / (1024 * 1000.0)))
print("[INFO] testImage_canny matrix: {:.4f}MB".format(
    (testImage_canny.nbytes) / (1024 * 1000.0)))

  0%|          | 0/158 [00:00<?, ?it/s]

(2, 1024)


100%|██████████| 158/158 [00:01<00:00, 114.02it/s]

(158, 1024)
[INFO] trainImage_canny matrix: 0.01MB
[INFO] testImage_canny matrix: 0.6320MB


In [10]:
del trainImage_canny
del testImage_canny

In [7]:
trainImage_canny_gray = batch_canny_gray(trainImage)
testImage_canny_gray = batch_canny_gray(testImage)

scaler = sklearn.preprocessing.StandardScaler().fit(trainImage_canny_gray)
trainImage_canny_gray = scaler.transform(trainImage_canny_gray)
testImage_canny_gray = scaler.transform(testImage_canny_gray)

np.save('./trainImage_canny_gray.npy', trainImage_canny_gray)
np.save('./testImage_canny_gray.npy', testImage_canny_gray)

print("[INFO] trainImage_canny_gray matrix: {:.2f}MB".format(
    (trainImage_canny_gray.nbytes) / (1024 * 1000.0)))
print("[INFO] testImage_canny_gray matrix: {:.4f}MB".format(
    (testImage_canny_gray.nbytes) / (1024 * 1000.0)))

  7%|▋         | 11/158 [00:00<00:01, 104.33it/s]

(2000, 1024)


100%|██████████| 158/158 [00:01<00:00, 125.11it/s]

(158, 1024)
[INFO] trainImage_canny_gray matrix: 8.00MB
[INFO] testImage_canny_gray matrix: 0.6320MB


In [8]:
del trainImage_canny_gray
del testImage_canny_gray

In [13]:
trainImage_GLCM = batch_GLCM(trainImage)
testImage_GLCM = batch_GLCM(testImage)

scaler = sklearn.preprocessing.StandardScaler().fit(trainImage_GLCM)
trainImage_GLCM = scaler.transform(trainImage_GLCM)
testImage_GLCM = scaler.transform(testImage_GLCM)

np.save('./trainImage_GLCM.npy', trainImage_GLCM)
np.save('./testImage_GLCM.npy', testImage_GLCM)

print("[INFO] trainImage_GLCM matrix: {:.2f}MB".format(
    (trainImage_GLCM.nbytes) / (1024 * 1000.0)))
print("[INFO] testImage_GLCM matrix: {:.4f}MB".format(
    (testImage_GLCM.nbytes) / (1024 * 1000.0)))

  1%|          | 1/158 [00:00<00:22,  6.95it/s]

(2, 12)


100%|██████████| 158/158 [00:22<00:00,  6.89it/s]

(158, 12)
[INFO] trainImage_GLCM matrix: 0.00MB
[INFO] testImage_GLCM matrix: 0.0074MB


In [14]:
del trainImage_GLCM
del testImage_GLCM

In [9]:
trainImage_pyramid = batch_pyramid(trainImage)
testImage_pyramid = batch_pyramid(testImage)

scaler = sklearn.preprocessing.StandardScaler().fit(trainImage_pyramid)
trainImage_pyramid = scaler.transform(trainImage_pyramid)
testImage_pyramid = scaler.transform(testImage_pyramid)

np.save('./trainImage_pyramid.npy', trainImage_pyramid)
np.save('./testImage_pyramid.npy', testImage_pyramid)

print("[INFO] trainImage_pyramid matrix: {:.2f}MB".format(
    (trainImage_pyramid.nbytes) / (1024 * 1000.0)))
print("[INFO] testImage_cpyramid matrix: {:.4f}MB".format(
    (testImage_pyramid.nbytes) / (1024 * 1000.0)))

  8%|▊         | 13/158 [00:00<00:01, 129.52it/s]

(2000, 1280)


100%|██████████| 158/158 [00:01<00:00, 121.14it/s]

(158, 1280)
[INFO] trainImage_pyramid matrix: 10.00MB
[INFO] testImage_cpyramid matrix: 0.7900MB


In [10]:
del trainImage_pyramid
del testImage_pyramid

In [22]:
# scaler = sklearn.preprocessing.StandardScaler().fit(trainImage)
# trainImage = scaler.transform(trainImage)
# testImage = scaler.transform(testImage)

np.save('./trainImage.npy', trainImage)
np.save('./testImage.npy', testImage)

print("[INFO] trainImage matrix: {:.2f}MB".format(
    (trainImage.nbytes) / (1024 * 1000.0)))
print("[INFO] testImage matrix: {:.2f}MB".format(
    (testImage.nbytes) / (1024 * 1000.0)))

[INFO] trainImage matrix: 0.38MB
[INFO] testImage matrix: 30.34MB


In [23]:
trainImage_canny = np.load('./trainImage_canny.npy')
testImage_canny = np.load('./testImage_canny.npy')
trainImage_GLCM = np.load('./trainImage_GLCM.npy')
testImage_GLCM = np.load('./testImage_GLCM.npy')

trainImage_canny_GLCM = np.hstack((trainImage_canny,trainImage_GLCM))
testImage_canny_GLCM = np.hstack((testImage_canny,testImage_GLCM))

scaler = sklearn.preprocessing.StandardScaler().fit(trainImage_canny_GLCM)
trainImage_canny_GLCM = scaler.transform(trainImage_canny_GLCM)
testImage_canny_GLCM = scaler.transform(testImage_canny_GLCM)

np.save('./trainImage_canny_GLCM.npy', trainImage_canny_GLCM)
np.save('./testImage_canny_GLCM.npy', testImage_canny_GLCM)

print("[INFO] trainImage_canny_GLCM matrix: {:.2f}MB".format(
    (trainImage_canny_GLCM.nbytes) / (1024 * 1000.0)))
print("[INFO] testImage_canny_GLCM matrix: {:.2f}MB".format(
    (testImage_canny_GLCM.nbytes) / (1024 * 1000.0)))

del trainImage_canny
del testImage_canny
del trainImage_GLCM
del testImage_GLCM
del trainImage_canny_GLCM
del testImage_canny_GLCM

[INFO] trainImage_canny_GLCM matrix: 0.01MB
[INFO] testImage_canny_GLCM matrix: 0.64MB


In [26]:
import keras
from keras.applications.vgg16 import VGG16,preprocess_input
#from keras.applications.vgg19 import VGG19,preprocess_input
#from keras.applications.inception_v3 import InceptionV3,preprocess_input
from keras.preprocessing import image

# grab the list of images that we'll be describing
print("[INFO] handling images...")
TRAIN_ORIGINAL_DIR = '../train/'
TRAIN_SUB_DIR = '../subsample/'
TRAIN_GAN = '../image_gan/'
TEST_DIR = '../test/'

# use this for full dataset
train_images = [TRAIN_SUB_DIR + i for i in os.listdir(TRAIN_SUB_DIR)]
test_images = [TEST_DIR + i for i in os.listdir(TEST_DIR)]

train_images.sort(key=natural_keys)
test_images.sort(key=natural_keys)

# initialize the features matrix and labels list
trainImage_CNN = []
testImage_CNN  = []

model = VGG16(weights='imagenet', include_top=False)
#model = InceptionV3(weights='imagenet', include_top=False)

# loop over the input images
for (i, imagePath) in enumerate(train_images):

    # extract CNN features in the image
    img = image.load_img(imagePath, target_size=(224,224)) #VGG16
    #img = image.load_img(imagePath, target_size=(299,299))#InceptionV3
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features = model.predict(x)[0, ...].flatten()

    # add the messages we got to features and labels matricies
    trainImage_CNN.append(features)

#     # show an update every 200 images until the last image
    if i > 0 and ((i + 1) % 200 == 0 or i == len(train_images) - 1):
        print("[INFO] processed {}/{}".format(i + 1, len(train_images)))


# loop over the test images
for (i, imagePath) in enumerate(test_images):
   
    # extract CNN features in the image
    img = image.load_img(imagePath, target_size=(224,224)) #VGG16
    #img = image.load_img(imagePath, target_size=(299,299))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features = model.predict(x)[0, ...].flatten()

    # add the messages we got to features and labels matricies
    testImage_CNN.append(features)

#     # show an update every 200 images until the last image
    if i > 0 and ((i + 1) % 100 == 0 or i == len(test_images) - 1):
        print("[INFO] processed {}/{}".format(i + 1, len(test_images)))


[INFO] handling images...
[INFO] processed 2/2
[INFO] processed 100/158
[INFO] processed 158/158


In [27]:
trainImage_CNN = np.array(trainImage_CNN,dtype = float32)
testImage_CNN = np.array(testImage_CNN,dtype = float32)

scaler = sklearn.preprocessing.StandardScaler().fit(trainImage_CNN)
trainImage_CNN = scaler.transform(trainImage_CNN)
testImage_CNN = scaler.transform(testImage_CNN)

print("[INFO] CNN Feature matrix: {:.2f}MB".format(
    (trainImage_CNN.nbytes + testImage_CNN.nbytes) / (1024 * 1000.0)))

#save matrix
np.save('./trainImage_CNN.npy', trainImage_CNN)
np.save('./testImage_CNN.npy', testImage_CNN)

[INFO] CNN Feature matrix: 15.68MB
